In [1]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard

In [2]:
import glob
import gc

import torch
import xarray as xr
from neuralhydrology.nh_run import start_run, eval_run
from scripts.file_manipulator import file_rewriter

from pathlib import Path

import random
random.seed(42)
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')
ts_dir = Path('../geo_data/time_series')

Using device: cuda

NVIDIA GeForce RTX 3080 Ti
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


### Run for different configs


#### ERA5 precipitation

##### train

In [3]:
file_rewriter(q_pathes=glob.glob('../geo_data/great_db/nc_all_q/*.nc'),
              ts_dir=ts_dir,
              hydro_target='q_cms_s',
              meteo_predictors=['t_max_e5l', 't_min_e5l', 'prcp_e5'])
if torch.cuda.is_available():
    start_run(config_file=Path("./configs/era5_prcp_qms.yml"))

/workspaces/my_dissertation/test_meteo_input/scripts/file_manipulator.py:16: RuntimeWarning: divide by zero encountered in true_divide
  condition_2 = max([max(value/ds_file[hydro_target].to_numpy())
/workspaces/my_dissertation/test_meteo_input/scripts/file_manipulator.py:16: RuntimeWarning: invalid value encountered in true_divide
  condition_2 = max([max(value/ds_file[hydro_target].to_numpy())
/workspaces/my_dissertation/test_meteo_input/scripts/file_manipulator.py:16: RuntimeWarning: divide by zero encountered in true_divide
  condition_2 = max([max(value/ds_file[hydro_target].to_numpy())
/workspaces/my_dissertation/test_meteo_input/scripts/file_manipulator.py:16: RuntimeWarning: invalid value encountered in true_divide
  condition_2 = max([max(value/ds_file[hydro_target].to_numpy())
/workspaces/my_dissertation/test_meteo_input/scripts/file_manipulator.py:16: RuntimeWarning: divide by zero encountered in true_divide
  condition_2 = max([max(value/ds_file[hydro_target].to_numpy())
/w

2023-04-02 14:34:48,082: Logging to runs_q_cms/era5_prcp_0204_143448/output.log initialized.
2023-04-02 14:34:48,082: ### Folder structure created at runs_q_cms/era5_prcp_0204_143448
2023-04-02 14:34:48,083: ### Run configurations for era5_prcp
2023-04-02 14:34:48,083: experiment_name: era5_prcp
2023-04-02 14:34:48,083: run_dir: runs_q_cms/era5_prcp_0204_143448
2023-04-02 14:34:48,084: train_basin_file: openf_basins.txt
2023-04-02 14:34:48,084: validation_basin_file: openf_basins.txt
2023-04-02 14:34:48,084: test_basin_file: openf_basins.txt
2023-04-02 14:34:48,085: train_start_date: 2008-01-01 00:00:00
2023-04-02 14:34:48,085: train_end_date: 2016-12-31 00:00:00
2023-04-02 14:34:48,085: validation_start_date: 2017-01-01 00:00:00
2023-04-02 14:34:48,086: validation_end_date: 2018-12-31 00:00:00
2023-04-02 14:34:48,086: test_start_date: 2019-01-01 00:00:00
2023-04-02 14:34:48,086: test_end_date: 2020-12-31 00:00:00
2023-04-02 14:34:48,087: per_basin_train_periods_file: None
2023-04-02 1

/opt/conda/lib/python3.9/site-packages/neuralhydrology/datasetzoo/basedataset.py:460: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1666643016022/work/torch/csrc/utils/tensor_new.cpp:230.)
  per_basin_target_stds = torch.tensor([np.nanstd(obs, axis=1)], dtype=torch.float32)


100%|██████████| 970/970 [00:00<00:00, 1682.72it/s]
2023-04-02 14:35:09,461: Create lookup table and convert to pytorch tensor
100%|██████████| 970/970 [00:13<00:00, 74.20it/s]
2023-04-02 14:35:22,536: These basins do not have a single valid sample in the train period: ['2106', '49135', '5664', '71277']
# Epoch 1: 100%|██████████| 11034/11034 [09:20<00:00, 19.70it/s, Loss: 0.0000]
2023-04-02 14:44:45,282: Epoch 1 average loss: 0.000219878975983442
# Epoch 2: 100%|██████████| 11034/11034 [09:31<00:00, 19.31it/s, Loss: 0.0000]
2023-04-02 14:54:16,691: Epoch 2 average loss: 7.912604801787595e-06
# Epoch 3: 100%|██████████| 11034/11034 [09:22<00:00, 19.61it/s, Loss: 0.0000]
2023-04-02 15:03:39,425: Epoch 3 average loss: 5.152022366751072e-06
# Validation: 100%|██████████| 970/970 [01:59<00:00,  8.15it/s]
2023-04-02 15:05:38,633: Stored results at runs_q_cms/era5_prcp_0204_143448/validation/model_epoch003/validation_results.p
2023-04-02 15:05:38,649: Epoch 3 average validation loss: 0.00000

##### test

In [4]:
run_dir = Path("./runs_q_cms/era5_prcp_0204_143448/")
eval_run(run_dir=run_dir, period="test")

2023-04-02 21:12:22,546: Using the model weights from runs_q_cms/era5_prcp_0204_143448/model_epoch042.pt
# Evaluation: 100%|██████████| 970/970 [02:05<00:00,  7.70it/s]
2023-04-02 21:14:28,589: Stored results at runs_q_cms/era5_prcp_0204_143448/test/model_epoch042/test_results.p


#### ERA5-Land precipitation

##### train

In [ ]:
file_rewriter(q_pathes=glob.glob('../geo_data/great_db/nc_all_q/*.nc'),
              ts_dir=ts_dir,
              hydro_target='q_cms_s',
              meteo_predictors=['t_max_e5l', 't_min_e5l', 'prcp_e5l'])
if torch.cuda.is_available():
    start_run(config_file=Path("./configs/era5_land_prcp_qms.yml"))

##### test

In [5]:
run_dir = Path("./runs_q_cms/era5_land_prcp_2803_125025")
eval_run(run_dir=run_dir, period="test")

# Evaluation: 100%|██████████| 1136/1136 [02:15<00:00,  8.40it/s]


#### GPCP precipitation

##### train

In [ ]:
# gpcp
file_rewriter(q_pathes=glob.glob('../geo_data/great_db/nc_all_q/*.nc'),
              ts_dir=ts_dir,
              hydro_target='q_cms_s',
              meteo_predictors=['t_max_e5', 't_min_e5', 'prcp_gpcp'])
if torch.cuda.is_available():
    start_run(config_file=Path("./configs/gpcp_prcp_qms.yml"))
gc.collect()

##### test

In [6]:
run_dir = Path("./runs_q_cms/gpcp_prcp_3003_063422")
eval_run(run_dir=run_dir, period="test")

# Evaluation: 100%|██████████| 1136/1136 [02:14<00:00,  8.43it/s]


#### IMERG precipitation

##### train

In [ ]:
file_rewriter(q_pathes=glob.glob('../geo_data/great_db/nc_all_q/*.nc'),
              ts_dir=ts_dir,
              hydro_target='q_cms_s',
              meteo_predictors=['t_max_e5', 't_min_e5', 'prcp_imerg'])
if torch.cuda.is_available():
    start_run(config_file=Path("./configs/imerg_prcp_qms.yml"))
gc.collect()

##### test

In [ ]:
run_dir = Path("./runs_q_cms/imerg_prcp_3003_135405")
eval_run(run_dir=run_dir, period="test")

#### MSWEP precipitation

##### train

In [ ]:
file_rewriter(q_pathes=glob.glob('../geo_data/great_db/nc_all_q/*.nc'),
              ts_dir=ts_dir,
              hydro_target='q_cms_s',
              meteo_predictors=['t_max_e5', 't_min_e5', 'prcp_mswep'],
              possible_nans=1)
if torch.cuda.is_available():
    start_run(config_file=Path("./configs/mswep_prcp_qms.yml"))
gc.collect()

##### test

In [8]:
run_dir = Path("./runs_q_cms/mswep_prcp_3103_073833")
eval_run(run_dir=run_dir, period="test")

# Evaluation: 100%|██████████| 1136/1136 [02:14<00:00,  8.46it/s]
